In [ ]:
# import tensorflow as tf
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import warnings

tf.reset_default_graph()

train = pd.read_csv("./data/mnist2/train.csv")
test = pd.read_csv("./data/mnist2/test.csv")


x_data=train.drop("label", axis=1, inplace=False)
train2=train["label"]



nn=int(len(x_data))

X=tf.placeholder(shape=[None,784], dtype=tf.float32)
Y=tf.placeholder(shape=[None,10], dtype=tf.float32)

keep_rate=tf.placeholder(dtype=tf.float32)  #드랍아웃에서 얼마나 살릴지


df1=pd.DataFrame()
arr = np.zeros((10,10), dtype=np.int32)

df1=arr

df2=pd.DataFrame()


for n in range(0,10):
    df1[n,n]=1
    df2[n]=np.arange(10)

tr2=int(len(train2))


for n in range(0,tr2):
    df2[n]=df1[train2[n]]

y_data=pd.DataFrame()

y_data=df2.transpose()


number=int(len(x_data)*2/3)

x_traindata=x_data[0:number]
x_testdata=x_data[number:-1]

y_traindata=y_data[0:number]
y_testdata=y_data[number:-1]



X_img = tf.reshape(X, shape=[-1,28,28,1])

W1=tf.Variable(tf.random_normal([5,5,1,32],stddev=0.01))

L1 = tf.nn.conv2d(X_img,W1,strides=[1,1,1,1], padding="SAME")

L1 = tf.nn.relu(L1)
p1 = tf.nn.max_pool(L1 , ksize=[1,2,2,1] , strides=[1,2,2,1], padding="SAME")

L2=tf.layers.conv2d(inputs=p1, filters=64, kernel_size=[5,5] , padding="SAME", strides=1 ,activation=tf.nn.relu)



L2=tf.layers.max_pooling2d(inputs=L2 , pool_size=[2,2], padding="SAME" , strides=2)

#########CONVOLUTION LAYER 끝

#4. NURAL NETWORK

L2=tf.reshape(L2,shape=[-1,7*7*64])

#print(L2.shape)

# 5. weight & bias

W2=tf.get_variable("weight2" , shape=[7*7*64,256], initializer=tf.contrib.layers.xavier_initializer())
b2=tf.Variable(tf.random_normal([256]),name="bias2" )

_layer1 = tf.nn.relu(tf.matmul(L2,W2)+b2 )
layer1 = tf.nn.dropout(_layer1 , keep_prob=keep_rate)


W3=tf.get_variable("weight3" , shape=[256,256], initializer=tf.contrib.layers.xavier_initializer())
b3=tf.Variable(tf.random_normal([256]),name="bias3" )

_layer2 = tf.nn.relu(tf.matmul(layer1,W3)+b3 )
layer2 = tf.nn.dropout(_layer2 , keep_prob=keep_rate)


W4=tf.get_variable("weight4" , shape=[256,10], initializer=tf.contrib.layers.xavier_initializer())
b4=tf.Variable(tf.random_normal([10]),name="bias4" )

# hypothesis 가설만들어야함


H = tf.matmul(layer2,W4)+b4

#cost 함수

cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=H , labels=Y))

train = tf.train.AdamOptimizer(learning_rate=0.000833).minimize(cost)

sess=tf.Session()
sess.run(tf.global_variables_initializer())

#학습
num_of_epoch = 10
batch_size=100

for step in range(10):
    num_of_iter = 42000
    print(step)
    n=0
    n1=100
    n2=100
    
    for i in range(280):
        batch_x = x_traindata[n:n2]
        batch_y = y_traindata[n:n2]
        _,cost_val=sess.run([train,cost], feed_dict={X:batch_x , Y:batch_y ,keep_rate:0.5
                                                   })
        
        n=n+n1
        n2=n2+n1
    print("cost : {}".format(cost_val))
    

predict = tf.argmax(H,1)
correct=tf.equal(predict , tf.argmax(Y,1)) 
accuracy=tf.reduce_sum(tf.cast(correct , dtype=tf.float32))

###테스트

# result_sum=0
# num_of_iter = int(num_of_iter/100)

# n=0
# n1=100
# n2=100

# for i in range(140):
#     batch_x=x_testdata[n:n2]
#     batch_y =y_testdata[n:n2]
#     accuracy_num=sess.run(accuracy, feed_dict={X:batch_x , Y:batch_y ,keep_rate:1})
#     result_sum += accuracy_num
#     n=n+n1
#     n2=n2+n1

# print("정확도:{}".format(result_sum/14000))

### 라벨 뽑기 predict 이용


n=0
n1=100
n2=100
a=[]

for i in range(280):
    batch_x=test[n:n2]

    #yy_value.loc[n:n2,"Label"]=sess.run(predict, feed_dict={X:batch_x,keep_rate:1})
    a.extend(sess.run(predict, feed_dict={X:batch_x,keep_rate:1}))
    
    n=n+n1
    n2=n2+n1

print("=========")

yy_value=pd.DataFrame(columns=["ImageId","Label"])

yy_value["Label"]=a

nnn=[]
for nnnn in range(1,28001):
    nnn.append(nnnn)

yy_value["ImageId"]=nnn
print(yy_value)


yy_value.to_csv("./data/submit_mnistwow280", index=False)


